In [2]:
import os
import json
from flask import Flask, request, jsonify
# from amplify import Amplify
from PIL import Image
import requests
import base64

In [3]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/heatherwang/Documents/Python/myvenv/WIT-BootCamp-Accenture-2024


In [ ]:
class MockAmplify:
    class API:
        @staticmethod
        def post(api_name, path, body):
            # Mock image generation
            return {"image": "base64_encoded_image_data"}

Amplify = MockAmplify()

In [ ]:
app = Flask(__name__)


# Initialize Amplify
# Amplify.configure({
#     "API": {
#         "endpoints": [
#             {
#                 "name": "imageGenerationApi",
#                 "endpoint": "YOUR_API_ENDPOINT_HERE"
#             }
#         ]
#     }
# })

# Local cache directory
CACHE_DIR = "image_cache"

def get_cached_image(text):
    """Check if an image for the given text exists in the local cache."""
    filename = f"{CACHE_DIR}/{base64.b64encode(text.encode()).decode()}.png"
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return base64.b64encode(f.read()).decode()
    return None

def save_to_cache(text, image_data):
    """Save the generated image to the local cache."""
    if not os.path.exists(CACHE_DIR):
        os.makedirs(CACHE_DIR)
    filename = f"{CACHE_DIR}/{base64.b64encode(text.encode()).decode()}.png"
    with open(filename, "wb") as f:
        f.write(base64.b64decode(image_data))

def generate_image(text):
    """Call the API to generate an image for the given text."""
    # response = Amplify.API.post(
    #     "imageGenerationApi",
    #     "/generate",
    #     body={"text": text}
    # )
    # img = response["image"]

    # dummy implementation
    img = base64.b64encode(b"image data for " + text.encode()).decode()
    return img

@app.route("/get_image", methods=["POST"])
def get_image():
    data = request.json
    text = data.get("text")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    # Check local cache
    cached_image = get_cached_image(text)
    if cached_image:
        return jsonify({"image": cached_image, "source": "cache"})

    # Generate new image
    try:
        generated_image = generate_image(text)
        save_to_cache(text, generated_image)
        return jsonify({"image": generated_image, "source": "api"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True)